
Hi, FlORence ici 🇫🇷

I'm a manager of AmazOR that wants to find the best possible locations for our warehouses while deciding how to allocate our client demands to these warehouses.

Our warehouses can only handle a limited amount of demand but we need to ensure that all client demands are met.

While opening a new facility, we have some fixed costs, and we also take into account the cost of allocating the demand of each client to each warehouse.

Of course, we want to minimize our costs here...

Can you help me solve this problem?

## Solution modeled with binary variables

**Goal**

Minimizes total cost of opening warehouses and serving demand to clients.

$$ \min \sum_n f_ny_n + \sum_{n,i}c_{ni}y_n$$

**Decisions**:

$y_i \in \{0,1\}$ decides to open warehouse $i$ and $x_{ij} \in \mathbb{R}_0^+$ assigns client $j$ to be served by warehouse $i$

**Data:**

Fixed costs for opening warehouse $i$ is given by $c_i$

Normalized costs for shipping one unit of demand from warehouse $i$ to client $j$ is given by $s_{ni}$  (can be computed from the data)

Set of warehouses $W=\{1..16\}$

Set of clients $C=\{1..50\}$

Maximal amount that can be served by warehouse $i$ is given by $u_i$

Demand required by customer $j$ is given by $d_j$ 

**Constraints**

| **Description** | **Expression** |
| --- | --- |
| Satisfy each customer’s demand  | $\forall j \in C: \sum_i x_{ij} = d_j$ |
| Stay within the warehouse’s capacity | $\forall i \in W: \sum_j x_{ij} \leq y_i u_i$ |

In [2]:
def get_instance(input_filename):
    with open(input_filename, "r") as file:
        data = file.read()

    # Split the data into lines
    lines = data.strip().split('\n')
    l = 0

    # Skip comment lines
    while lines[l].startswith('#') or lines[l] == '':
        l += 1
    
    # get metadata parameters (always first line)
    metadata = map(int, lines[l].split())
    data = []
    l += 1

    # Process each line
    while l < len(lines):
        data.append(list(map(float, lines[l].split())))
        l += 1

    return *metadata, data

def parse_to_zero_index(data, indexes):
    for i in range(len(data)):
        for idx in indexes:
            data[i][idx] -= 1
    return

In [3]:
def process_data(raw_data, num_warehouses, num_customers):
    wh_capacity, wh_opening_cost = [], []
    for _ in range(num_warehouses):
        capacity, opening_cost = raw_data.pop(0)
        wh_capacity.append(int(capacity))
        wh_opening_cost.append(opening_cost)

    demand, sending_cost = [], []
    for _ in range(num_customers):
        d = raw_data.pop(0)[0]
        demand.append(int(d))
        sending_cost.append([cost/d for cost in raw_data.pop(0)]) # calculate cost of sending one item from warehouse

    return wh_capacity, wh_opening_cost, demand, sending_cost

In [4]:
num_warehouses, num_customers, raw_data = get_instance("data/warehouse_costs.txt")
wh_capacity, wh_opening_cost, demand, sending_cost = process_data(raw_data, num_warehouses, num_customers)

# Print the extracted data
print(f"Number of warehouses: {num_warehouses}")
print(f"Number of customers: {num_customers}")
print(f"Sending cost matrix: {len(sending_cost), len(sending_cost[0])}")

Number of warehouses: 16
Number of customers: 50
Sending cost matrix: (50, 16)


In [5]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

In [6]:
y = [None] * num_warehouses
x = [[None] * num_warehouses for _ in range(num_customers)]

for w in range(num_warehouses):
    y[w] = model.NewBoolVar(f'is_warehouse_{w}_open')
    for c in range(num_customers):
        x[c][w] = model.NewIntVar(0, 10000, f'shipping_amount_to_{c}_by_{w}')

In [7]:
# 1. Satisfy customer demand
for c in range(num_customers):
    model.Add(sum(x[c][w] for w in range(num_warehouses)) == demand[c])

In [8]:
# 2. Subject to warehouse capacities
for w in range(num_warehouses):
    model.Add(sum(x[c][w] for c in range(num_customers)) <= y[w] * wh_capacity[w])

In [9]:
# Specify the type of problem. In this case, we want to minimize the objective function
solver.parameters.num_search_workers = 8
solver.parameters.max_time_in_seconds = 120
model.Minimize(
    sum(wh_opening_cost[w] * y[w]
        for w in range(num_warehouses)
    ) + \
    sum(sending_cost[c][w] * x[c][w] 
        for c in range(num_customers) 
        for w in range(num_warehouses)
    )
)

In [16]:
# Call the solver method to find the optimal solution
callback = cp_model.ObjectiveSolutionPrinter()
or_status = solver.SolveWithSolutionCallback(model, callback)
status = solver.StatusName(or_status)

if status in ["OPTIMAL", "FEASIBLE"]:
    print(f'Solution: Total cost = {solver.ObjectiveValue()}')
else:
    print('A solution could not be found, check the problem specification')

Solution 0, time = 0.04 s, objective = 2050422
Solution 1, time = 0.05 s, objective = 1040444
Solution: Total cost = 1040444.3749999999


In [17]:
import pandas as pd

solution = []
for w in range(num_warehouses):
    for c in range(num_customers):
        if solver.Value(x[c][w]) > 0:
            solution.append((w, c, solver.Value(x[c][w])))

df = pd.DataFrame(solution, columns=['warehouse_id', 'customer_id', 'shipping_amount'])
# [solver.Value(x[s][v]) for v in range(num_cities) if x[s][v] is not None]

In [18]:
import plotly.graph_objects as go

fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = df['shipping_amount'],
                   colorscale = 'OrRd',
                   showscale = True,
                   cmin = 0,
                   cmax = 7500),
        dimensions = list([
            dict(range = [0,num_warehouses-1],
                #  constraintrange = [3,5],
                 label = "warehouse_id", values = df['warehouse_id']),
            dict(range = [0,num_customers-1],
                 constraintrange = [32.9,33.1],
                 label = 'customer_id', values = df['customer_id'])])
    )
)
fig.show()
